## Allen NLP NER + Anonymization

### Importing Libraries

In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import gensim

In [2]:
import joblib

### Loading Allen NLP NER model

In [3]:
predictor_ner = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo.2021-02-12.tar.gz")

/Users/divyam/Semester 7/Project Elective - Prof Vinu/Web app/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['vectors', 'textcat', 'parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [4]:
predictor_ner

In [5]:
data = "Albert Einstein was born in Germany and lived in England."

In [6]:
results = predictor_ner.predict(sentence=data)

In [7]:
lst = []

for word, tag in zip(results['words'], results['tags']):
  tpl = (word, tag)
  lst.append(tpl) 

In [8]:
final_lst = []
i = 0
n = len(lst)
s = ""
prev_tag = ""
while i < n:
  ps = lst[i][0]
  pt = lst[i][1]

  if pt[0] == 'B':
    if s != "":
      tpl = (s, prev_tag)
      final_lst.append(tpl)
    s = ps
    prev_tag = pt[2:]

  elif pt[0] == 'I': 
    s = s + " " + ps
    prev_tag = pt[2:]

  elif pt[0] == 'L':
    s = s + " " + ps
    tpl = (s, pt[2:])
    final_lst.append(tpl)
    s=""

  elif pt[0] == 'U':
    if s!="":
      tpl = (s, prev_tag)
      final_lst.append(tpl)
    tpl1 = (ps, pt[2:])
    final_lst.append(tpl1)
    s=""
    prev_tag=""

  elif pt == "O":
    if(prev_tag == "O"):
      if ps == '.' or ps == ',':
        s = s+ps
      else:
        s = s + " " + ps
    else:
      if s!="":
        tpl = (s, prev_tag)
        final_lst.append(tpl)
      s = ps
    prev_tag = pt

  i+=1

if s!="":
  tpl = (s, prev_tag)
  final_lst.append(tpl)
      


In [9]:
print(final_lst)

[('Albert Einstein', 'PER'), ('was born in', 'O'), ('Germany', 'LOC'), ('and lived in', 'O'), ('England', 'LOC'), ('.', 'O')]


In [10]:
print(lst)

[('Albert', 'B-PER'), ('Einstein', 'L-PER'), ('was', 'O'), ('born', 'O'), ('in', 'O'), ('Germany', 'U-LOC'), ('and', 'O'), ('lived', 'O'), ('in', 'O'), ('England', 'U-LOC'), ('.', 'O')]


In [11]:
para = ""

for item in final_lst:
  if para == "":
    para = item[0]
  else:
    para = para + " " + item[0]

In [12]:
para

'Albert Einstein was born in Germany and lived in England .'

# Post-Midterm

In [13]:
final_str = ""
final_words = []
tag_of_intrest = "LOC"
idxs_of_tags = []

In [14]:
i = 0
for tpl in final_lst:
  word = ""
  if tpl[1] != 'O':
    word = tpl[0].replace(' ', '_')
    final_str = final_str + " " + word
  else:
    word = tpl[0]
    final_str = final_str + " " + word
  if tpl[1] == tag_of_intrest:
    idxs_of_tags.append(i)

  final_words.append(word)
  i+=1

final_str = final_str[1:]

In [15]:
print(final_str)
print(final_words)
print(idxs_of_tags)

Albert_Einstein was born in Germany and lived in England .
['Albert_Einstein', 'was born in', 'Germany', 'and lived in', 'England', '.']
[2, 4]


In [16]:
model = gensim.models.Word2Vec.load("../Word2vec/w2vpandas.model")

In [17]:
look_up = {}

top_n = 20

In [18]:
for i in idxs_of_tags:
  lst = []
  print(i)
  w = final_words[i]
  keys = list(look_up.keys())
  values = list(look_up.values())
  if w in keys:
    print('b')
    final_words[i] = look_up[w]
  else:
    lst = model.wv.most_similar(w, topn=top_n)
    j=0
    while j<top_n:
      if lst[i][0] in values:
        j+=1
        continue
      else:
        break

    if j<top_n:
      final_words[i] = lst[j][0]
      look_up[w] = lst[j][0]

2
4


In [19]:
look_up

{'Germany': 'Hammerschmidt_Villa', 'England': 'Mobage'}

In [20]:
print(final_words)

['Albert_Einstein', 'was born in', 'Hammerschmidt_Villa', 'and lived in', 'Mobage', '.']


In [21]:
anonymized_string = ""

for w in final_words:
  if anonymized_string == "":
    anonymized_string = w
  else:
    anonymized_string = anonymized_string + " " + w

In [22]:
print(anonymized_string)

Albert_Einstein was born in Hammerschmidt_Villa and lived in Mobage .


In [23]:
output = anonymized_string.replace('_', ' ')

In [24]:
output

'Albert Einstein was born in Hammerschmidt Villa and lived in Mobage .'

In [25]:
joblib.dump(model, "./word2vec.joblib", compress=True)

['./word2vec.joblib']

In [26]:
w2vmodel = joblib.load("./word2vec.joblib")

In [27]:
final_str = ""
final_words = []
tag_of_intrest = "LOC"
idxs_of_tags = []

In [28]:
i = 0
for tpl in final_lst:
  word = ""
  if tpl[1] != 'O':
    word = tpl[0].replace(' ', '_')
    final_str = final_str + " " + word
  else:
    word = tpl[0]
    final_str = final_str + " " + word
  if tpl[1] == tag_of_intrest:
    idxs_of_tags.append(i)

  final_words.append(word)
  i+=1

final_str = final_str[1:]

In [29]:
print(final_str)
print(final_words)
print(idxs_of_tags)

Albert_Einstein was born in Germany and lived in England .
['Albert_Einstein', 'was born in', 'Germany', 'and lived in', 'England', '.']
[2, 4]


In [30]:
look_up = {}

top_n = 20

In [31]:
for i in idxs_of_tags:
  lst = []
  w = final_words[i]
  keys = list(look_up.keys())
  values = list(look_up.values())
  if w in keys:
    final_words[i] = look_up[w]
  else:
    lst = w2vmodel.wv.most_similar(w, topn=top_n)
    j=0
    while j<top_n:
      if lst[i][0] in values:
        j+=1
        continue
      else:
        break

    if j<top_n:
      final_words[i] = lst[j][0]
      look_up[w] = lst[j][0]

In [32]:
look_up

{'Germany': 'Hammerschmidt_Villa', 'England': 'Mobage'}

In [33]:
print(final_words)

['Albert_Einstein', 'was born in', 'Hammerschmidt_Villa', 'and lived in', 'Mobage', '.']


In [34]:
anonymized_string = ""

for w in final_words:
  if anonymized_string == "":
    anonymized_string = w
  else:
    anonymized_string = anonymized_string + " " + w

In [35]:
print(anonymized_string)

Albert_Einstein was born in Hammerschmidt_Villa and lived in Mobage .


In [36]:
output = anonymized_string.replace('_', ' ')

In [37]:
output

'Albert Einstein was born in Hammerschmidt Villa and lived in Mobage .'